In [103]:
!pip install scipy
import pandas as pd
import scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.9/29.9 MB 7.2 MB/s eta 0:00:00m eta 0:00:010:00:01


In [104]:
df = pd.read_csv(
    "/tmp/reviews/amazon_reviews_us_Digital_Video_Download_v1_00.tsv.gz",
    delimiter="\t",
    error_bad_lines=False,
)
df = df[["customer_id", "product_id", "star_rating", "product_title"]]
customers = df["customer_id"].value_counts()
products = df["product_id"].value_counts()

quantiles = [
    0,
    0.1,
    0.25,
    0.5,
    0.75,
    0.8,
    0.85,
    0.9,
    0.95,
    0.96,
    0.97,
    0.98,
    0.99,
    0.995,
    0.999,
    0.9999,
    1,
]
customers = customers[customers >= 35]
products = products[products >= 20]

reduced_df = df.merge(pd.DataFrame({"customer_id": customers.index})).merge(
    pd.DataFrame({"product_id": products.index})
)

customers = reduced_df["customer_id"].value_counts()
products = reduced_df["product_id"].value_counts()

test_products = products.sample(frac=0.005)
train_products = products[~(products.index.isin(test_products.index))]

customer_index = pd.DataFrame(
    {"customer_id": customers.index, "user": np.arange(customers.shape[0])}
)
train_product_index = pd.DataFrame(
    {"product_id": train_products.index, "item": np.arange(train_products.shape[0])}
)
test_product_index = pd.DataFrame(
    {"product_id": test_products.index, "item": np.arange(test_products.shape[0])}
)

train_df = reduced_df.merge(customer_index).merge(train_product_index)
test_df = reduced_df.merge(customer_index).merge(test_product_index)

train_sparse = scipy.sparse.csr_matrix(
    (
        np.where(train_df["star_rating"].values >= 4, 1, 0),
        (train_df["item"].values, train_df["user"].values),
    ),
    shape=(train_df["item"].nunique(), customers.count()),
)

test_sparse = scipy.sparse.csr_matrix(
    (
        np.where(test_df["star_rating"].values >= 4, 1, 0),
        (test_df["item"].values, test_df["user"].values),
    ),
    shape=(test_df["item"].nunique(), customers.count()),
)

np.savetxt("/tmp/reviews/train.csv", train_sparse.todense(), delimiter=",", fmt="%i")

np.savetxt("/tmp/reviews/test.csv", test_sparse.todense(), delimiter=",", fmt="%i")


train_s3 = sagemaker_session.upload_data(
    "/tmp/reviews/train.csv", bucket=bucket, key_prefix="{}/pca/train".format(prefix)
)

test_s3 = sagemaker_session.upload_data(
    "/tmp/reviews/test.csv", bucket=bucket, key_prefix="{}/pca/test".format(prefix)
)

/var/folders/1b/_jbv1lxn457d4z71n10rh0sc0000gn/T/ipykernel_92894/2129752244.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(
b'Skipping line 92523: expected 15 fields, saw 22\n'
b'Skipping line 343254: expected 15 fields, saw 22\n'
b'Skipping line 524626: expected 15 fields, saw 22\n'
b'Skipping line 623024: expected 15 fields, saw 22\n'
b'Skipping line 977412: expected 15 fields, saw 22\n'
b'Skipping line 1496867: expected 15 fields, saw 22\n'
b'Skipping line 1711638: expected 15 fields, saw 22\n'
b'Skipping line 1787213: expected 15 fields, saw 22\n'
b'Skipping line 2395306: expected 15 fields, saw 22\n'
b'Skipping line 2527690: expected 15 fields, saw 22\n'


NameError: name 'sess' is not defined

In [106]:
train_s3

's3://sagemaker-han-batch/tutorial/pca/train/train.csv'

In [52]:
# 初期設定

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models

import os, boto3, json, sagemaker, numpy as np
from io import BytesIO

def make_dir(path):
    if os.path.isdir(path):
        pass
    else:
        os.mkdir(path)
        
sagemaker_session = sagemaker.Session()

# AWS設定
role = 'han_s3_full_access'
region = boto3.Session().region_name
bucket='sagemaker-han-batch'
prefix = 'tutorial'
bucket_path = 'https://s3-{}.amazonaws.com/{}'.format(region,bucket)

# Local設定
data_dir = './tutorial'
model_dir = os.path.join(data_dir, 'model')
input_dir = os.path.join(data_dir, 'inputs')
output_dir = os.path.join(data_dir, 'outputs')
code_dir = os.path.join(data_dir, 'code')
for dir_name in [data_dir, model_dir, input_dir, output_dir, code_dir]:
    make_dir(dir_name)

In [17]:
# Create temp model and 

In [53]:
%%writefile {model_dir}/my_model.txt
Hello my great machine learning model

Writing ./tutorial/model/my_model.txt


In [64]:
! ls

Pipfile                           batch_transform_image_input.ipynb
Pipfile.lock                      requirements.txt
Untitled.ipynb                    src
batch-transform-tutorial.ipynb    train.csv.out
batch_transform_1.ipynb           tutorial


In [65]:
# Upload into S3
%cd {model_dir}
!tar zcvf model.tar.gz ./*
%cd ../..

model_s3_uri = sagemaker.session.Session().upload_data(
    f'{model_dir}/model.tar.gz',
    bucket = bucket,
    key_prefix = 'tutorial_model'
)
print(model_s3_uri)

/Users/han/Desktop/Fusic/2022/aws_batch_transform/tutorial/model
a ./model.tar.gztar: ./model.tar.gz: Can't add archive to itself

a ./my_model.txt
/Users/han/Desktop/Fusic/2022/aws_batch_transform
s3://sagemaker-han-batch/tutorial_model/model.tar.gz


In [66]:
%%writefile {code_dir}/inference.py
import os

def model_fn(model_dir):
    with open(os.path.join(model_dir,'my_model.txt')) as f:
        model = f.read()[:-1] # 改行を除外
    return model

def predict_fn(input_data, model):
    response = f'{model} for the {input_data}st time'
    return response

Overwriting ./tutorial/code/inference.py


In [40]:
# モデルとコンテナの指定

from sagemaker.pytorch import PyTorchModel

model_name = 'TxtModel'
model_data = model_s3_uri
endpoint_name = model_name + 'Endpoint'

pytorch_model = PyTorchModel(
    name = model_name,
    model_data = model_data,
    role = role,
    framework_version = '1.12.0',
    py_version='py38',
    entry_point='inference.py',
    source_dir = code_dir
)

pytorch_predictor = pytorch_model.deploy(
    initial_instance_count=1,
    instance_type='ml.m4.xlarge',
    enpoint_name=endpoint_name
)

-------!

In [42]:
response = pytorch_predictor.predict(1)
print(response,type(response))

Hello my great machine learning model for the 1st time <class 'numpy.ndarray'>


In [43]:
pytorch_predictor.delete_endpoint()

In [41]:
container_image_uri = sagemaker.image_uris.retrieve(
    "pytorch", 
    sagemaker.session.Session().boto_region_name, # ECR のリージョンを指定
    version='1.11.0', # SKLearn のバージョンを指定
    instance_type = 'ml.m4.large', # インスタンスタイプを指定
    image_scope = 'inference' # 推論コンテナを指定
)
print(container_image_uri)

763104351884.dkr.ecr.us-west-2.amazonaws.com/pytorch-inference:1.11.0-cpu-py38


In [113]:
%%writefile {code_dir}/inference1.py
import logging
import os, json

logger = logging.getLogger()

def model_fn(model_dir):
    with open(os.path.join(model_dir,'my_model.txt')) as f:
        hello = f.read()[:-1] # 改行を除外
    return hello

def input_fn(input_data, content_type):
    logger.info(len(input_data)
#     if content_type == 'text/csv':
#         transformed_data = input_data.split(',')
#     else:
#         raise ValueError(f"Illegal content type {content_type}. The only allowed content_type is text/csv")
    return list(input_data)

def predict_fn(transformed_data, model):
    prediction_list = []
    logger.info(len(transformed_data))
#     for data in transformed_data:
#         if data[-1] == '1':
#             ordinal = f'{data}st'
#         elif data[-1] == '2':
#             ordinal = f'{data}nd'
#         elif data[-1] == '3':
#             ordinal = f'{data}rd'
#         else:
#             ordinal = f'{data}th'
#         prediction = f'{model} for the {ordinal} time'
#         prediction_list.append(prediction)   
    return len(transformed_data)
def output_fn(prediction_list, accept):
#     if accept == 'text/csv':    
#         response = ''
#         for prediction in prediction_list:
#             response += prediction + '\n'
#     else:
#         raise ValueError(f"Illegal accept type {accept}. The only allowed accept type is text/csv")
    return prediction_list, accept

Overwriting ./tutorial/code/inference1.py


In [114]:
%cd {model_dir}
!rm model.tar.gz
!cp ../../{code_dir}/inference1.py ./
!tar zcvf model.tar.gz ./*
%cd ../..

/Users/han/Desktop/Fusic/2022/aws_batch_transform/tutorial/model
a ./inference.py
a ./inference1.py
a ./my_model.txt
/Users/han/Desktop/Fusic/2022/aws_batch_transform


In [115]:
model_s3_uri = sagemaker.session.Session().upload_data(
    f'{model_dir}/model.tar.gz',
    bucket = bucket,
    key_prefix = 'tutorial_model'
)
print(model_s3_uri)

s3://sagemaker-han-batch/tutorial_model/model.tar.gz


In [94]:
# Batch Transform

input_data1 = 'input_data1.csv'
input_data2 = 'input_data2.csv'
long_input = ','.join([str(i) for i in range(99999)])
with open(os.path.join(input_dir,input_data1),'wt') as f:
    f.write(long_input)
with open(os.path.join(input_dir,input_data2),'wt') as f:
    f.write('11,22,33,44,55')

input_prefix = prefix + '/input'
input_data_s3_uri = sagemaker.Session().upload_data(
    input_dir,
    bucket = bucket,
    key_prefix = input_prefix)
print(input_data_s3_uri)

s3://sagemaker-han-batch/tutorial/input


In [116]:
max_concurrent_transforms = 1
max_payload = 1
strategy = 'MultiRecord'# 'SingleRecord'

data_type = "S3Prefix"
split_type = "Line" # "Line"



model_data = 's3://sagemaker-us-west-2-582981179587/TxtModel/model.tar.gz'
pytorch_model = PyTorchModel(model_data = model_data,
                             entry_point='inference1.py',
                             source_dir = code_dir,
                             framework_version='1.12.0',
                             py_version='py38',
                             role = role)

output_s3_path = 's3://sagemaker-han-batch/tutorial/output'
transformer = pytorch_model.transformer(instance_count=1, 
                                        instance_type="ml.m5.large",
                                        output_path=output_s3_path,
                                        accept = 'text/csv',
                                        max_concurrent_transforms = max_concurrent_transforms,
                                        max_payload = max_payload,
                                        strategy = strategy
                                       )

In [117]:
transformer.transform(
    data=train_s3,
    data_type = data_type,
    split_type = split_type,
    content_type = 'text/csv',
    wait=True,
)

2022-08-22T06:57:13,318 [INFO ] main org.pytorch.serve.servingsdk.impl.PluginsManager - Initializing plugins manager...
2022-08-22T06:57:13,455 [INFO ] main org.pytorch.serve.ModelServer - 
Torchserve version: 0.6.0
TS Home: /opt/conda/lib/python3.8/site-packages
Current directory: /
Temp directory: /home/model-server/tmp
Number of GPUs: 0
Number of CPUs: 2
Max heap size: 980 M
Python executable: /opt/conda/bin/python3.8
Config file: /etc/sagemaker-ts.properties
Inference address: http://0.0.0.0:8080
Management address: http://0.0.0.0:8080
Metrics address: http://127.0.0.1:8082
Model Store: /.sagemaker/ts/models
Initial Models: model=/opt/ml/model
Log dir: /logs
Metrics dir: /logs
Netty threads: 0
Netty client threads: 0
Default workers per model: 2
Blacklist Regex: N/A
Maximum Response Size: 6553500
Maximum Request Size: 6553500
Limit Maximum Image Pixels: true
Prefer direct buffer: false
Allowed Urls: [file://.*|http(s)?://.*]
Custom python dependency for model allowed: false
Metrics

2022-08-22T06:57:17,849 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - Listening on port: /home/model-server/tmp/.ts.sock.9000
2022-08-22T06:57:17,851 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - [PID]99
2022-08-22T06:57:17,852 [INFO ] W-9000-model_1.0 org.pytorch.serve.wlm.WorkerThread - Connecting to: /home/model-server/tmp/.ts.sock.9000
2022-08-22T06:57:17,852 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - Torch worker started.
2022-08-22T06:57:17,853 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - Python runtime: 3.8.13
2022-08-22T06:57:17,869 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - Connection accepted: /home/model-server/tmp/.ts.sock.9000.
2022-08-22T06:57:17,869 [INFO ] W-9000-model_1.0 org.pytorch.serve.wlm.WorkerThread - Flushing req. to backend at: 1661151437869
2022-08-22T06:57:17,882 [INFO ] W-9001-model_1.0-stdout MODEL_LOG - Listening on port: /home/model-server/tmp/.ts.sock.9001
2022-08-22T06:57:17,883 [INFO ] W-9001-model_1.0-stdout MODEL_LOG - [PID]102
2022-08-22T06:57:17,884 

2022-08-22T06:57:28.871:[sagemaker logs]: MaxConcurrentTransforms=1, MaxPayloadInMB=1, BatchStrategy=MULTI_RECORD
2022-08-22T06:57:33,451 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - Listening on port: /home/model-server/tmp/.ts.sock.9000
2022-08-22T06:57:33,452 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - [PID]217
2022-08-22T06:57:33,452 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - Torch worker started.
2022-08-22T06:57:33,453 [INFO ] W-9000-model_1.0 org.pytorch.serve.wlm.WorkerThread - Connecting to: /home/model-server/tmp/.ts.sock.9000
2022-08-22T06:57:33,451 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - Listening on port: /home/model-server/tmp/.ts.sock.9000
2022-08-22T06:57:33,452 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - [PID]217
2022-08-22T06:57:33,452 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - Torch worker started.
2022-08-22T06:57:33,453 [INFO ] W-9000-model_1.0 org.pytorch.serve.wlm.WorkerThread - Connecting to: /home/model-server/tmp/.ts.sock.9000
2022-08-22T06:57:33,453 [INF

2022-08-22T06:57:34,095 [INFO ] W-9000-model_1.0-stderr org.pytorch.serve.wlm.WorkerLifeCycle - Stopped Scanner - W-9000-model_1.0-stderr
2022-08-22T06:57:34,096 [INFO ] W-9000-model_1.0-stdout org.pytorch.serve.wlm.WorkerLifeCycle - Stopped Scanner - W-9000-model_1.0-stdout
2022-08-22T06:57:34,188 [INFO ] W-9001-model_1.0-stderr org.pytorch.serve.wlm.WorkerLifeCycle - Stopped Scanner - W-9001-model_1.0-stderr
2022-08-22T06:57:42,449 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - Listening on port: /home/model-server/tmp/.ts.sock.9000
2022-08-22T06:57:42,450 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - [PID]239
2022-08-22T06:57:42,450 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - Torch worker started.
2022-08-22T06:57:42,450 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - Python runtime: 3.8.13
2022-08-22T06:57:42,450 [INFO ] W-9000-model_1.0 org.pytorch.serve.wlm.WorkerThread - Connecting to: /home/model-server/tmp/.ts.sock.9000
2022-08-22T06:57:42,451 [INFO ] W-9000-model_1.0 org.pytorch.serve

2022-08-22T06:57:56,431 [INFO ] W-9000-model_1.0 org.pytorch.serve.wlm.WorkerThread - Connecting to: /home/model-server/tmp/.ts.sock.9000
2022-08-22T06:57:56,431 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - Python runtime: 3.8.13
2022-08-22T06:57:56,433 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - Connection accepted: /home/model-server/tmp/.ts.sock.9000.
2022-08-22T06:57:56,433 [INFO ] W-9000-model_1.0 org.pytorch.serve.wlm.WorkerThread - Flushing req. to backend at: 1661151476433
2022-08-22T06:57:56,434 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - model_name: model, batchSize: 1
2022-08-22T06:57:56,469 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - Backend worker process died.
2022-08-22T06:57:56,470 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - Traceback (most recent call last):
2022-08-22T06:57:56,470 [INFO ] W-9000-model_1.0-stdout MODEL_LOG -   File "/opt/conda/lib/python3.8/site-packages/ts/model_service_worker.py", line 210, in <module>
2022-08-22T06:57:56,472 [INFO ] W-9000-model_1.0-

...

2022-08-22T07:01:19,356 [INFO ] W-9001-model_1.0-stdout MODEL_LOG -   File "/opt/conda/lib/python3.8/site-packages/ts/model_service_worker.py", line 139, in handle_connection
2022-08-22T07:01:19,357 [INFO ] W-9001-model_1.0-stdout MODEL_LOG -     service, result, code = self.load_model(msg)
2022-08-22T07:01:19,357 [INFO ] W-9001-model_1.0-stdout MODEL_LOG -   File "/opt/conda/lib/python3.8/site-packages/ts/model_service_worker.py", line 104, in load_model
2022-08-22T07:01:19,357 [INFO ] W-9001-model_1.0-stdout MODEL_LOG -     service = model_loader.load(
2022-08-22T07:01:19,357 [INFO ] W-9001-model_1.0-stdout MODEL_LOG -   File "/opt/conda/lib/python3.8/site-packages/ts/model_loader.py", line 151, in load
2022-08-22T07:01:19,357 [INFO ] W-9001-model_1.0-stdout MODEL_LOG -     initialize_fn(service.context)
2022-08-22T07:01:19,357 [INFO ] W-9001-model_1.0-stdout MODEL_LOG -   File "/opt/conda/lib/python3.8/site-packages/sagemaker_pytorch_serving_container/handler_service.py", line 51, i

2022-08-22T07:03:44,074 [INFO ] W-9001-model_1.0-stdout MODEL_LOG -     user_module = importlib.import_module(user_module_name)
2022-08-22T07:03:44,074 [INFO ] W-9001-model_1.0-stdout MODEL_LOG -   File "/opt/conda/lib/python3.8/importlib/__init__.py", line 127, in import_module
2022-08-22T07:03:44,074 [INFO ] W-9001-model_1.0-stdout MODEL_LOG -     return _bootstrap._gcd_import(name[level:], package, level)
2022-08-22T07:03:44,074 [INFO ] W-9001-model_1.0-stdout MODEL_LOG -   File "<frozen importlib._bootstrap>", line 1014, in _gcd_import
2022-08-22T07:03:44,074 [INFO ] W-9001-model_1.0-stdout MODEL_LOG -   File "<frozen importlib._bootstrap>", line 991, in _find_and_load
2022-08-22T07:03:44,075 [INFO ] W-9001-model_1.0-stdout MODEL_LOG -   File "<frozen importlib._bootstrap>", line 975, in _find_and_load_unlocked
2022-08-22T07:03:44,075 [INFO ] W-9001-model_1.0-stdout MODEL_LOG -   File "<frozen importlib._bootstrap>", line 671, in _load_unlocked
2022-08-22T07:03:44,075 [INFO ] W-900

2022-08-22T07:07:38,231 [INFO ] W-9001-model_1.0-stdout org.pytorch.serve.wlm.WorkerLifeCycle - Stopped Scanner - W-9001-model_1.0-stdout
2022-08-22T07:07:38,231 [INFO ] W-9001-model_1.0-stderr org.pytorch.serve.wlm.WorkerLifeCycle - Stopped Scanner - W-9001-model_1.0-stderr
2022-08-22T07:13:54,565 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - Torch worker started.
2022-08-22T07:13:54,565 [INFO ] W-9000-model_1.0 org.pytorch.serve.wlm.WorkerThread - Connecting to: /home/model-server/tmp/.ts.sock.9000
2022-08-22T07:13:54,571 [INFO ] W-9000-model_1.0 org.pytorch.serve.wlm.WorkerThread - Flushing req. to backend at: 1661152434571
2022-08-22T07:13:54,576 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - Python runtime: 3.8.13
2022-08-22T07:13:54,576 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - Connection accepted: /home/model-server/tmp/.ts.sock.9000.
2022-08-22T07:13:54,577 [INFO ] W-9000-model_1.0-stdout MODEL_LOG - model_name: model, batchSize: 1
2022-08-22T07:13:54,610 [INFO ] W-9000-model_1.0-

2022-08-22T07:13:55,449 [INFO ] W-9001-model_1.0 org.pytorch.serve.wlm.WorkerThread - Connecting to: /home/model-server/tmp/.ts.sock.9001
2022-08-22T07:13:55,450 [INFO ] W-9001-model_1.0 org.pytorch.serve.wlm.WorkerThread - Flushing req. to backend at: 1661152435450
2022-08-22T07:13:55,451 [INFO ] W-9001-model_1.0-stdout MODEL_LOG - Connection accepted: /home/model-server/tmp/.ts.sock.9001.
2022-08-22T07:13:55,451 [INFO ] W-9001-model_1.0-stdout MODEL_LOG - model_name: model, batchSize: 1
2022-08-22T07:13:55,485 [INFO ] W-9001-model_1.0-stdout MODEL_LOG - Backend worker process died.
2022-08-22T07:13:55,485 [INFO ] W-9001-model_1.0-stdout MODEL_LOG - Traceback (most recent call last):
2022-08-22T07:13:55,485 [INFO ] W-9001-model_1.0-stdout MODEL_LOG -   File "/opt/conda/lib/python3.8/site-packages/ts/model_service_worker.py", line 210, in <module>
2022-08-22T07:13:55,485 [INFO ] W-9001-model_1.0-stdout MODEL_LOG -     worker.run_server()
2022-08-22T07:13:55,485 [INFO ] W-9001-model_1.0

KeyboardInterrupt: 